In [ ]:
#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py


import tensorflow 
from tensorflow.keras.optimizers import SGD       
from tensorflow.keras.models import Sequential 
from tensorflow.keras.applications import VGG16   #VGG16 pretrained weights    
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D


# to make this notebook's output stable across runs
np.random.seed(42)
tensorflow.random.set_seed(42)
#loading the dataset
train_dataset = h5py.File('/cxldata/datasets/project/cat-non-cat/train_catvnoncat.h5', "r")
test_dataset = h5py.File('/cxldata/datasets/project/cat-non-cat/test_catvnoncat.h5', "r")

print("File format of train_dataset:",train_dataset)
print("File format of test_dataset:",test_dataset)

# Load training data
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

# Load test data
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels
#visualizing the data
classes = np.array(test_dataset["list_classes"][:])

print("Classes are: ",classes)
print("Groundtruth stored as: ",train_set_y_orig[2])
print(classes[train_set_y_orig[2]].decode('utf-8'))

plt.imshow(train_set_x_orig[2])
#shape of the data
validation_x = test_set_x_orig[:25] 
validation_y = test_set_y_orig[:25]

test_set_x = test_set_x_orig[25:]
test_set_y = test_set_y_orig[25:]

print("train_set_x shape: ", train_set_x_orig.shape)
print("train_set_y shape: ", train_set_y_orig.shape)

print("Validation data size: ",validation_x.shape)
print("Validation data size: ", validation_y.shape)

print("test_set_x shape: ", test_set_x.shape)
print("test_set_y shape: ", test_set_y.shape)
#getting the pre trained vgg16 model
vgg_base = VGG16(weights='imagenet', include_top=False)

vgg_base.trainable=False
#customizing  the vgg16 base model
inp = Input(shape=(64, 64, 3), name='image_input')

#initiate a model
vgg_model = Sequential() #Add the VGG base model
vgg_model.add(vgg_base)

vgg_model.add(GlobalAveragePooling2D())

vgg_model.add(Dense(1024,activation='relu'))
vgg_model.add(Dropout(0.6))

vgg_model.add(Dense(512,activation='relu'))
vgg_model.add(Dropout(0.5))

vgg_model.add(Dense(1024,activation='relu'))
vgg_model.add(Dropout(0.4))

vgg_model.add(Dense(1024,activation='relu'))
vgg_model.add(Dropout(0.3))

vgg_model.add(Dense(1, activation='sigmoid'))
#compile the vgg16 model
sgd = SGD(lr=0.025)
vgg_model.compile(loss='binary_crossentropy', optimizer=sgd,    metrics=['accuracy'])
# training the top layers of the model
vgg_model.fit(train_set_x_orig, train_set_y_orig, epochs=10, verbose=1, validation_data=(validation_x, validation_y))
#evaluating the model
vgg_model_loss, vgg_model_acc = vgg_model.evaluate(test_set_x_orig,test_set_y_orig)
print('Test accuracy using VGG16 model as base:', vgg_model_acc)
#visualizing the model
vgg_base.summary()

from tensorflow.keras.utils import plot_model
plot_model(vgg_base, show_shapes=True, show_layer_names=True)

vgg_model.summary()
plot_model(vgg_model, show_shapes=True, show_layer_names=True)